In [33]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
--2021-03-23 10:00:23--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.78.228
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.78.228|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18357730 (18M) [application/x-gzip]
Saving to: ‘./data/amazon_electronics_reviews_training.json.gz’

     0K .......... .......... .......... .......... ..........  0% 3.91M 4s
    50K .......... .......... .......... .......... ..........  0% 7.12M 3s
   100K .......... .......... .......... .......... ..........  0% 8.41M 3s
   150K .......... .......... .......... .......... ..........  1% 23.8M 2s
   200K .......... .......... .......... .......... ..........  1% 35.8M 2s
   250K .......... .......... .......... .......... ..........  1% 41.7M 2s
  

In [5]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [7]:
type(data), len(data)

(list, 60000)

In [8]:
data[42]

{'reviewerID': 'A254S5CCQGWEKO',
 'asin': 'B00009R6WO',
 'reviewerName': 'ROGERWILCO357 "ROGERWILCO357"',
 'helpful': [1, 2],
 'reviewText': "I have to say for the money you get a lot considering the 16-35mm at double the money love the lens for it's landscape properties",
 'overall': 5.0,
 'summary': 'love it',
 'unixReviewTime': 1312934400,
 'reviewTime': '08 10, 2011'}

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [9]:
ratings = [item["overall"] for item in data]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import Ridge

class KeySelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, Y=None):
        return self
    
    def transform(self, X):
        return[d[self.key] for d in X]

In [12]:
ks = KeySelector('reviewText')
x_trans = ks.fit_transform(data)
print(x_trans[0])

I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatically supports wheel mice.  Just plug it in and use it - it's that easy.


In [16]:
bag_of_words_model = Pipeline([
    ('KeySelector', KeySelector('reviewText')),
    ('Vectorizer', HashingVectorizer()),
    ('RidgeRegressor', Ridge(alpha=0.42))
])

bag_of_words_model.fit(data, ratings);

In [17]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score:  1.8379354831572012


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [40]:
###### from sklearn.linear_model import Lasso

normalized_model = Pipeline([
    ('Selector', KeySelector('reviewText')),
    ('Vectorizer', TfidfVectorizer()),
    ('RidgeRegressor', Ridge(alpha=.64))
])

normalized_model.fit(data, ratings);

In [41]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score:  1.0422944158262102


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [27]:
bigrams_model = Pipeline([
    ('Selector', KeySelector('reviewText')),
    ('Vectorizer', HashingVectorizer(ngram_range=(1,2))),
    ('RidgeRegressor', Ridge(alpha=0.6))
])

bigrams_model.fit(data, ratings);

In [28]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  1.1596368687144936


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [32]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

--2021-03-23 10:13:24--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.67.220
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.67.220|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970853 (2.8M) [application/x-gzip]
Saving to: ‘./data/amazon_one_and_five_star_reviews.json.gz’

     0K .......... .......... .......... .......... ..........  1% 3.69M 1s
    50K .......... .......... .......... .......... ..........  3% 7.28M 1s
   100K .......... .......... .......... .......... ..........  5% 6.83M 0s
   150K .......... .......... .......... .......... ..........  6% 44.0M 0s
   200K .......... .......... .......... .......... ..........  8% 8.75M 0s
   250K .......... .......... .......... .......... .......... 10%  106M 0s
   300K .......... .......... .......... .......... .......

To avoid memory issue, we can delete the older data.

In [42]:
del data, ratings

In [43]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [item['overall'] for item in data_polarity]

In [44]:
pipe = Pipeline([
    ('Selector', KeySelector('reviewText')),
    ('Vectorizer', TfidfVectorizer(stop_words='english')),
    ('NaiveBayesClassifier', MultinomialNB())
])

pipe.fit(data_polarity, ratings);

In [45]:
#get features (vocab) from model
feat_to_token = pipe['Vectorizer'].get_feature_names()

#get the log probability from model
log_prob = pipe['NaiveBayesClassifier'].feature_log_prob_

#collapse log probability into one row
polarity = log_prob[0, :] - log_prob[1, :]

#combine polarity and feature names
most_polar = sorted(list(zip(polarity, feat_to_token)))

In [46]:
n=25
most_polar = most_polar[:n] + most_polar[-n:]

top_50 = [term for score, term in most_polar]

In [47]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*